## Zadanie domowe: morfologiczna gra w życie – John Conway

### Wykorzystanie operacji LUT w przekształceniu trafi, nie trafi
  - Szybszą metodą wykonania transformacji trafi, nie trafi może być operacja LUT.
  - Technika polega na zakodowaniu wyników wszystkich interesujących  konfiguracji, a następnie podczas przetwarzania wykorzystania operacji LUT.
  - Dla otoczenia 3x3 możliwe jest 512 różnych konfiguracji.
  - Aby praktycznie zrealizować operacje, każdej konfiguracji należy nadać unikalny indeks. Jedną z możliwości jest pomnożenie elementu strukturalnego przez macierz (mnożenie odpowiednich elementów):
  ```
  [[1, 8,  64],
   [ 2, 16, 128],
   [ 4, 32, 256]]
  ```
  Przykładowo elementowi:
  ```
  [[1, 1, 0],
   [ 1, 0, 1],
   [ 1, 0, 1]]
  ```
  odpowiada liczba: 1(1) + 2(1) + 4(1) + 8(1) + 128(1) + 256(1) = 399.
  
### Przykład działania metody – detekcja punktów końcowych na obrazie.
  - założenie: punkt końcowy to punkt, który ma dokładnie jednego sąsiada,
  - zdefiniuj funkcję, która jako argument pobiera otoczenie, a zwraca 0 lub 1 w zależności od tego, czy rozpatrywany punkt jest końcowy np. dla sąsiedztwa 3×3 punkt będzie końcowy, jeżeli jest zapalony i ma tylko jednego sąsiada (czyli suma pikseli jest równa 2).
  - wygeneruj przekodowanie LUT.
  - wczytaj obraz szkielet.bmp (należy go przekształcić, aby uzyskać dwuwymiarową tablicę o wartościach 0-1). Wykorzystując wygenerowane przekodowanie LUT wykonaj detekcję zakończeń. Wyświetl obraz oryginalny i po przekodowaniu LUT.

### Gra w życie

Reguły gry w życie:
  - każdy piksel biały, który ma dwóch lub trzech sąsiadów (białych) przeżywa,
  - każdy piksel biały, który ma 0,1 lub więcej niż trzech sąsiadów (białych) nie przeżywa (głód lub przeludnienie),
  - jeżeli jakieś pole (czarne) sąsiaduje dokładnie z trzema pikselami białymi, to na tym polu ,,rodzi'' się nowy piksel biały.

Zadanie:
  - za pomocą mechanizmu LUT (opisanego wcześniej) należy zaimplementować morfologiczną gre w życie,
  - najważniejszym elementem jest funkcja opisująca reguły gry,
  - symulacje należny przeprowadzić dla plansz dostarczonych w pliku gra.py,
  - dobrze jest wykonać kilka iteracji – zobaczyć jak zmienia się kształt,
  - inne ciekawe kształty do znalezienia w internecie.


In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

if not os.path.exists("szkielet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/10_Morfologia/szkielet.bmp --no-check-certificate

szkielet = cv2.imread('szkielet.bmp', cv2.IMREAD_GRAYSCALE)

plt.imshow(szkielet, 'gray')
plt.axis('off')
plt.show()

def bit_counter(n):
    counter = 0
    while n:
        n = n & (n - 1)
        counter += 1
    return counter

def end_point(O):
    M = np.array([[1, 8, 64], [2, 16, 128], [4, 32, 256]])
    O = O / 255
    O = O.astype(bool)
    n = np.sum(M, where = O)
    return lut[int(n)]


lut = np.zeros(512)
for i in range(0b001000000000):
    if (i & 0b000010000):
        if bit_counter(i) == 2:
            lut[i] = 1
    else:
        lut[i] = 0

X, Y = szkielet.shape
copy = np.zeros((X, Y))
for i in range(1, X - 1):
    for j in range(1, Y - 1):
        O = szkielet[i - 1 : i + 2, j - 1 : j + 2]
        point = end_point(O)
        copy[i, j] = point

plt.imshow(copy, 'gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()

In [ ]:
def przekodowania():
    lut = np.zeros(512)
    for i in range(0b001000000000):
        if (i & 0b000010000):
            if bit_counter(i) == 3 or bit_counter(i) == 4:
                lut[i] = 1
            else:
                lut[i] = 0
        else:
            if bit_counter(i) == 3:
                lut[i] = 1
            else:
                lut[i] = 0
    return lut


def is_alive(O, LUT):
    M = np.array([[1, 8, 64], [2, 16, 128], [4, 32, 256]])
    O = O.astype(bool)
    n = np.sum(M, where = O)
    return LUT[int(n)]

# "Zabka" oraz "Szybowiec"
plansza = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0],
    [0,1,0,1,0,0,0,0,0,0,0,0],
    [0,0,1,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,1,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,1,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0]], np.uint8)

X, Y = plansza.shape
plansza_old = plansza.copy()
plt.imshow(plansza, 'gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()
for k in range(24):
    plansza_new = np.zeros((X, Y))
    for i in range(1, X - 1):
        for j in range(1, Y - 1):
            O = plansza_old[i - 1 : i + 2, j - 1 : j + 2]
            point = is_alive(O, przekodowania())
            plansza_new[i, j] = point
    
    plt.imshow(plansza_new, 'gray', vmin=0, vmax=1)
    plt.axis('off')
    plt.show()
    plansza_old = plansza_new.copy()